In [2]:
import pandas as pd
import numpy as np
# from sklearn.mixture import GaussianMixture
import multiprocessing
from multiprocessing.pool import MapResult
from Utils import remove_splchars,clean

# Preprocessing

In [5]:
df = pd.read_parquet("paraquet/train.parquet", engine="fastparquet")
df.drop(["id"], axis=1, inplace=True)
print(len(df))

287113


In [6]:
def split(df, NUM_SPLITS: int =6):
    splits = ()
    split_range = len(df)//NUM_SPLITS
    for ind in range(NUM_SPLITS):
        splits+= (df.iloc[ind:ind+split_range,...],)
    return splits


cur_data = split(df)

In [7]:
#pool = multiprocessing.Pool()
with multiprocessing.Pool(7) as p:
    df["article"] = p.map(remove_splchars,df["article"])

In [10]:
df.loc[0,"article"]

" 1411 EST, 25 October 2013 . 1536 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda pictured of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month. Symptoms of hepatitis A in

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143557 entries, 0 to 143556
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   highlights  143557 non-null  object
 1   cleaned     143557 non-null  object
 2   tokens      143557 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [ ]:
df_size = df["cleaned"].size

for i in range (0, df_size):
    temp = df.loc[i,"cleaned"]
    tokenised = word_tokenize(temp, 'english', preserve_line=True)
    df["tokens"][i] = tokenised


In [ ]:
df.to_pickle("tokened.pkl")


def batching(df):
    batch_size = 1000
    temp = 0
    steps = np.ceil((df["cleaned"].size)/batch_size)
    while(steps>=0):
        df_temp = df.iloc[temp*batch_size:(temp+1)*batch_size, 1]
        yield df_temp
        temp+=1
        steps-=1 

count = 0

for i in batching(df):
    temp = i.transform(lambda x: word_tokenize(x, "english", preserve_line=True))
    # temp = word_tokenize(i, "english", preserve_line=True)
    df.iloc[count*1000:(count+1)*1000, 2] = temp 
    count += 1

In [66]:
df = pd.read_pickle("tokened.pkl")

In [ ]:
new_df = df.loc[0:10,...]
new_df.drop(["tokens"],axis=1,inplace=True)

In [74]:
df_size = df["cleaned"].size
new_df = pd.concat([new_df,df.loc[0:10,"tokens"].transform(lambda x: [spacy_object(token)[0].lemma_ for token in x ])],axis=1)

In [75]:
new_df

,highlights,cleaned,tokens
0,"Bishop John Folda, of North Dakota, is taking ...","By . Associated Press . PUBLISHED . 1411 EST, ...","[by, ., associate, press, ., PUBLISHED, ., 141..."
1,Criminal complaint: Cop used his role to help ...,CNN Ralph Mata was an internal affairs lieute...,"[CNN, Ralph, Mata, be, an, internal, affair, l..."
2,"Craig Eccleston-Todd, 27, had drunk at least t...",A drunk driver who killed a young woman in a h...,"[a, drunk, driver, who, kill, a, young, woman,..."
3,Nina dos Santos says Europe must be ready to a...,CNN With a breezy sweep of his pen President ...,"[CNN, with, a, breezy, sweep, of, his, pen, Pr..."
4,Fleetwood top of League One after 2-0 win at S...,Fleetwood are the only team still to have a 10...,"[fleetwood, be, the, only, team, still, to, ha..."
5,Prime Minister and his family are enjoying an ...,He's been accused of making many a fashion fau...,"[he, be, be, accuse, of, make, many, a, fashio..."
6,NBA star calls for black and Hispanic communit...,By . Daily Mail Reporter . PUBLISHED . 0115 ES...,"[by, ., Daily, mail, reporter, ., PUBLISHED, ...."
7,London Midland service had been pulling into T...,By . Daily Mail Reporter . This is the moment ...,"[by, ., Daily, mail, reporter, ., this, be, th..."
8,Tony Pulis believes Saido Berahino should look...,There are a number of job descriptions waiting...,"[there, be, a, number, of, job, description, w..."
9,Black box data from Flight 370 could be analyz...,"Canberra, Australia CNN At first glance, it d...","[Canberra, ,, Australia, CNN, at, first, glanc..."


In [76]:
new_df.to_pickle("pickeled_10")

# Glove

In [77]:
df = new_df

In [22]:
embedding_dict = {}

In [23]:
with open("glove/glove.6B.50d.txt","r",encoding="utf-8") as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:],dtype="float64")
        embedding_dict[word] = vector
        #break

In [35]:
print(list(embedding_dict.keys())[6])

in


In [82]:
for i in df.loc[2,"tokens"]:
    print(embedding_dict[i.lower()])

[ 0.21705   0.46515  -0.46757   0.10082   1.0135    0.74845  -0.53104
 -0.26256   0.16812   0.13182  -0.24909  -0.44185  -0.21739   0.51004
  0.13448  -0.43141  -0.03123   0.20674  -0.78138  -0.20148  -0.097401
  0.16088  -0.61836  -0.18504  -0.12461  -2.2526   -0.22321   0.5043
  0.32257   0.15313   3.9636   -0.71365  -0.67012   0.28388   0.21738
  0.14433   0.25926   0.23434   0.4274   -0.44451   0.13813   0.36973
 -0.64289   0.024142 -0.039315 -0.26037   0.12017  -0.043782  0.41013
  0.1796  ]
[-0.41524   -0.82576   -0.36691   -1.1605     0.1775     0.59261
 -0.14077    0.81616   -0.03875    0.57861    0.083448   0.063366
 -0.22196   -0.1286     1.2429    -0.5219    -0.49453    0.97288
 -0.34983   -1.1664     0.10412    0.98756    0.81013    1.1988
  0.44121   -1.5622     0.22322    0.77086    1.2683     0.17224
  1.0374     0.25958   -0.16946   -0.061829   0.56236   -0.0072596
  0.34725   -0.28291    0.15776    0.22653   -0.4861     0.45098
 -0.022293   0.50258    0.59848   -0.9037

KeyError: 'ecclestontodd'

# Word2Vec

In [87]:
words = list(df.loc[:,"tokens"])

In [89]:
wv_model = Word2Vec(words,min_count = 1,vector_size=50)

In [100]:
wv_model.most_similar(["associate"])

11

In [101]:
wv_model.corpus_total_words

7615

In [102]:
wv_model.corpus_count

11

In [ ]:
wv_model.wv.cosine_similarities(["associate"],vectors_all=True)

# TF-IDF

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [109]:
strings = list(df["cleaned"])

In [118]:
tfidf = TfidfVectorizer()
result = tfidf.fit_transform(strings)

In [126]:
vocab = tfidf.vocabulary_
names = list(vocab.keys())

In [142]:
for name, idf in zip(names,tfidf.idf_):
    print(name," "*(15-len(name)),"-->"," "*10,idf)

by               -->            2.791759469228055
associated       -->            2.791759469228055
press            -->            2.791759469228055
published        -->            2.791759469228055
1411             -->            2.791759469228055
est              -->            2.791759469228055
25               -->            2.386294361119891
october          -->            2.791759469228055
2013             -->            2.791759469228055
updated          -->            2.386294361119891
1536             -->            2.791759469228055
the              -->            2.791759469228055
bishop           -->            2.791759469228055
of               -->            2.791759469228055
fargo            -->            2.791759469228055
catholic         -->            2.791759469228055
diocese          -->            2.791759469228055
in               -->            2.791759469228055
north            -->            2.791759469228055
dakota           -->            2.791759469228055
